In [2]:
from openfisca_survey_manager.coicop import build_raw_coicop_nomenclature

In [4]:
raw_coicop_1998 = build_raw_coicop_nomenclature(year = 1998)

In [12]:
raw_coicop_1998['label_division'].unique()

array(['Produits alimentaires et boissons non alcoolisées',
       'Boissons alcoolisées et tabac', 'Habillement et chaussures',
       'Logement, eau, gaz, électricité et autres combustibles',
       'Ameublement, équipement ménager et entretien courant de la maison',
       'Santé', 'Transport', 'Communications', 'Loisirs et culture',
       'Éducation', 'Hôtellerie, cafés, restauration',
       'Autres biens et services'], dtype=object)

In [13]:
raw_coicop_1998.loc[raw_coicop_1998['label_division'] == 'Boissons alcoolisées et tabac']

,label_division,label_groupe,label_classe,label_sous_classe,label_poste,code_coicop
71,Boissons alcoolisées et tabac,Boissons alcoolisées,Alcools,Apéritifs,Apéritifs anisés à base d'alcool,2.1.1.1.1
72,Boissons alcoolisées et tabac,Boissons alcoolisées,Alcools,Apéritifs,Whisky,2.1.1.1.2
73,Boissons alcoolisées et tabac,Boissons alcoolisées,Alcools,Eaux de vie et liqueurs,Eaux de vie,2.1.1.2.1
74,Boissons alcoolisées et tabac,Boissons alcoolisées,Alcools,Eaux de vie et liqueurs,Liqueurs,2.1.1.2.2
75,Boissons alcoolisées et tabac,Boissons alcoolisées,"Vins, cidres et champagne",Vins,Vins de consommation courante,2.1.2.1.1
76,Boissons alcoolisées et tabac,Boissons alcoolisées,"Vins, cidres et champagne",Vins,Vins supérieurs,2.1.2.1.2
77,Boissons alcoolisées et tabac,Boissons alcoolisées,"Vins, cidres et champagne",Vins,Vins doux naturels et apéritifs à base de vin,2.1.2.1.3
78,Boissons alcoolisées et tabac,Boissons alcoolisées,"Vins, cidres et champagne","Champagne, mousseux et cidres",Champagne et mousseux,2.1.2.2.1
79,Boissons alcoolisées et tabac,Boissons alcoolisées,"Vins, cidres et champagne","Champagne, mousseux et cidres",Cidres,2.1.2.2.2
80,Boissons alcoolisées et tabac,Boissons alcoolisées,Bières,Bières,Bières,2.1.3.1.1


In [14]:
raw_coicop_2016 = build_raw_coicop_nomenclature(year = 2016)

In [16]:
raw_coicop_2016['label_division'].unique() 

array(['Produits alimentaires et boissons non alcoolisées',
       'Boissons alcoolisées, tabac et stupéfiants',
       "Articles d'habillement et chaussures",
       'Logement, eau, gaz, électricité et autres combustibles',
       'Meubles, articles de ménage et entretien courant du foyer',
       'Santé', 'Transports', 'Communications', 'Loisirs et culture',
       'Enseignement', 'Restaurants et hôtels',
       'Biens et services divers'], dtype=object)

In [17]:
raw_coicop_2016.loc[raw_coicop_2016['label_division'] == 'Boissons alcoolisées, tabac et stupéfiants']

,label_division,label_groupe,label_classe,label_sous_classe,label_poste,code_coicop
91,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Alcools de bouche,Spiritueux et liqueurs,Apéritifs anisés à base d'alcool,2.1.1.1.1
92,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Alcools de bouche,Spiritueux et liqueurs,Whisky,2.1.1.1.2
93,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Alcools de bouche,Spiritueux et liqueurs,Eaux de vie autres que le whisky,2.1.1.1.3
94,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Alcools de bouche,Spiritueux et liqueurs,Liqueurs et autres spiritueux n.c.a.,2.1.1.1.4
95,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Alcools de bouche,Boissons faiblement alcoolisées,Boissons faiblement alcoolisées,2.1.1.2.1
96,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Vin et boissons fermentées,Vin,Vins de consommation courante,2.1.2.1.1
97,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Vin et boissons fermentées,Vin,Vins supérieurs,2.1.2.1.2
98,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Vin et boissons fermentées,Vin,Champagne et mousseux,2.1.2.1.3
99,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Vin et boissons fermentées,Boissons fermentées à base d'autres fruits,Boissons fermentées à base d'autres fruits,2.1.2.2.2
100,"Boissons alcoolisées, tabac et stupéfiants",Boissons alcoolisées,Vin et boissons fermentées,Vins enrichis en alcool,Vins enrichis en alcool,2.1.2.3.1


In [7]:
from openfisca_france_indirect_taxation.scripts.build_coicop_nomenclature import build_complete_coicop_nomenclature

ImportError: cannot import name 'TypeIs' from 'typing_extensions' (c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\typing_extensions.py)